In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
from datetime import datetime
import pickle

import backtrader as bt
import pandas as pd

from polaristools.polarisbot import PolarisBot
from strategies import mystrategies

In [2]:
polaris = PolarisBot()

# Define functions

In [3]:
def optimization(
                symbol:str,
                timeframe:str,
                cash:int,
                sizer:int,
                comm:float,
                sample:dict,
                custom_strategy:mystrategies,
                parameters:dict
                ):
    cerebro = bt.Cerebro()
    
    cerebro.broker.set_cash( cash )
    cerebro.addsizer(bt.sizers.PercentSizer, percents=sizer)
    cerebro.broker.setcommission(commission=comm, mult=parameters.get('leverage_factor'))
    
    if timeframe == '1d':
        tframe = bt.TimeFrame.Days
    else:
        tframe = bt.TimeFrame.Minutes
    
    filename = f'df_klines_{symbol}_{timeframe}'
    df = polaris.dataframeFromBinary(filename)
    if sample:
        df = df.loc[sample.get('start'):sample.get('end')].copy()
    
    feed = bt.feeds.PandasData(dataname=df, timeframe=tframe, compression = int(timeframe[:-1]))
    cerebro.adddata(feed)
    
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    
    cerebro.optstrategy(custom_strategy, **parameters)
    backtests = cerebro.run()
    
    return backtests

In [4]:
def parse_analyzers(backtests):
    try:
        all_params = [dict(zip(
                list(x[0].params._getkeys()),
                list(x[0].params._getvalues()),
            ))for x in backtests]
    # except:
        # print('SOMETHING HAS FAILED WHEN TRY TO RETRIEVE THE PARAMETERS FROM STRATEGY.', __main__)
        df_params = pd.DataFrame(all_params)
        
        df_params.drop(columns=['verbose'], inplace=True)
        
        trades = [dict(
            pnl_net       = x[0].analyzers.tradeanalyzer.get_analysis()['pnl']['net']['total'], 
            trades  = x[0].analyzers.tradeanalyzer.get_analysis()['total']['total'], 
            won           = x[0].analyzers.tradeanalyzer.get_analysis()['won']['total'], 
            lost          = x[0].analyzers.tradeanalyzer.get_analysis()['lost']['total'],
            
            long_won      = x[0].analyzers.tradeanalyzer.get_analysis()['long']['won'],
            short_won     = x[0].analyzers.tradeanalyzer.get_analysis()['short']['won'],
            
            long_lost     = x[0].analyzers.tradeanalyzer.get_analysis()['long']['lost'],
            short_lost    = x[0].analyzers.tradeanalyzer.get_analysis()['short']['lost'],
            
            longs_pnl    = x[0].analyzers.tradeanalyzer.get_analysis()['long']['pnl']['total'],
            shorts_pnl    = x[0].analyzers.tradeanalyzer.get_analysis()['short']['pnl']['total'],
            
            moneydown_max = x[0].analyzers.drawdown.get_analysis()['max']['moneydown'],
        )for x in backtests]
        
        df_trades = pd.DataFrame(trades)
        df_results = pd.concat([df_params, df_trades], axis=1)
        
        return df_results
    except:
        print('FAILED PARSED')
        return pd.DataFrame()

In [5]:
def filter_results(dataframe, symbol:str, timeframe:str, by_col:str):
    if dataframe.empty:
        print(f'FAILED TEST ON symbol: {symbol}, tf: {timeframe}')
        return dataframe
    labels = ['pnl_net','won','lost',
            'trades','moneydown_max',
            'long_won','long_lost',
            'short_won','short_lost',
            'longs_pnl','shorts_pnl',
            'leverage_factor','aroon_timeperiod','ema',]
    best5 = dataframe[labels].nlargest(5, by_col)
    worse3 = dataframe[labels].nsmallest(3, by_col)
    
    bs = f'{symbol} {timeframe} BEST_5' 
    ws = f'{symbol} {timeframe} WORSE_3'
    best_horse = pd.concat([best5,worse3], keys=[bs, ws], axis=0)
    
    return best_horse

# LOOP #20 - RUN Multiple tests

In [6]:
# def loop_optimizations(symbols:list, timeframes:list, sample:dict, parameters:dict):

symbols    = ['BTCUSDT','BNBUSDT', 'DOGEUSDT', 'ETHUSDT']   #len 4
timeframes = ['240m', '120m', '60m', '30m', '15m']          #len 5
sample     = {'start':'2021-01-01', 'end':'2022-08-12'}

parameters = dict(
                    enter_long          = True,
                    enter_short         = True,
                    
                    ema                 = list(range(50, 101, 10)),
                    aroon_timeperiod    = list(range(50, 101, 10)),
                    leverage_factor     = 1.0,
)

# Store each results Dataframe here.
symbols_df = pd.DataFrame()
idx20=0
for symbol in symbols:
    for timeframe in timeframes:
        backtest = optimization(
                                symbol          = symbol,
                                timeframe       = timeframe,
                                cash            = 100,
                                sizer           = 25,
                                comm            = 0.05,
                                sample          = sample,
                                custom_strategy = mystrategies.AroonPlusMa,
                                parameters      = parameters,
                                )
        df = parse_analyzers(backtest)
        
        best5_worse3 = filter_results(dataframe=df, symbol=symbol, timeframe=timeframe, by_col='pnl_net')
        
        if symbols_df.empty:
            # First iteration check.
            symbols_df = best5_worse3
        else:
            symbols_df = pd.concat([symbols_df, best5_worse3], axis=0)
        
        idx20+=1
        now = datetime.now()
        print(f'ITERATION n# {idx20} at {now}. SYMBOL: {symbol} TIMEFRAME: {timeframe}')
        display(best5_worse3)

ITERATION n# 1 at 2022-08-14 03:26:35.197725. SYMBOL: BTCUSDT TIMEFRAME: 240m


pnl_net  won  lost  trades  moneydown_max  \
BTCUSDT - 240m BEST_5  8    4.351694    2     0       2      10.536115   
                       14   4.351694    2     0       2      10.536115   
                       20   4.351694    2     0       2      10.536115   
                       26   4.351694    2     0       2      10.536115   
                       32   4.351694    2     0       2      10.536115   
BTCUSDT - 240m WORSE_3 5  -20.417321    0     4       4      21.532423   
                       11 -20.417321    0     4       4      21.532423   
                       17 -20.417321    0     4       4      21.532423   

                           long_won  long_lost  short_won  short_lost  \
BTCUSDT - 240m BEST_5  8          2          0          0           0   
                       14         2          0          0           0   
                       20         2          0          0           0   
                       26         2          0          0           0   
                       32         2          0          0           0   
BTCUSDT - 240m WORSE_3 5          0          0          0           4   
                       11         0          0          0           4   
                       17         0          0          0           4   

                           longs_pnl  shorts_pnl  leverage_factor  \
BTCUSDT - 240m BEST_5  8    4.351694    0.000000              1.0   
                       14   4.351694    0.000000              1.0   
                       20   4.351694    0.000000              1.0   
                       26   4.351694    0.000000              1.0   
                       32   4.351694    0.000000              1.0   
BTCUSDT - 240m WORSE_3 5    0.000000  -20.417321              1.0   
                       11   0.000000  -20.417321              1.0   
                       17   0.000000  -20.417321              1.0   

                           aroon_timeperiod  ema  
BTCUSDT - 240m BEST_5  8                 70   60  
                       14                70   70  
                       20                70   80  
                       26                70   90  
                       32                70  100  
BTCUSDT - 240m WORSE_3 5                100   50  
                       11               100   60  
                       17               100   70

ITERATION n# 2 at 2022-08-14 03:27:06.621866. SYMBOL: BTCUSDT TIMEFRAME: 120m


pnl_net  won  lost  trades  moneydown_max  \
BTCUSDT - 120m BEST_5  21   0.419480    2     3       5      12.811925   
                       27   0.419480    2     3       5      12.811925   
                       3   -1.159290    2     4       6      12.187549   
                       9   -1.159290    2     4       6      12.187549   
                       15  -1.159290    2     4       6      12.187549   
BTCUSDT - 120m WORSE_3 12 -28.807194    1     8       9      30.463020   
                       30 -27.582277    1     6       7      29.238103   
                       0  -27.392750    1     8       9      30.797277   

                           long_won  long_lost  short_won  short_lost  \
BTCUSDT - 120m BEST_5  21         1          2          1           1   
                       27         1          2          1           1   
                       3          1          3          1           1   
                       9          1          3          1           1   
                       15         1          3          1           1   
BTCUSDT - 120m WORSE_3 12         1          5          0           3   
                       30         1          3          0           3   
                       0          1          8          0           0   

                           longs_pnl  shorts_pnl  leverage_factor  \
BTCUSDT - 120m BEST_5  21   2.022273   -1.602793              1.0   
                       27   2.022273   -1.602793              1.0   
                       3    0.443503   -1.602793              1.0   
                       9    0.443503   -1.602793              1.0   
                       15   0.443503   -1.602793              1.0   
BTCUSDT - 120m WORSE_3 12 -14.510565  -14.296629              1.0   
                       30 -13.285648  -14.296629              1.0   
                       0  -27.392750    0.000000              1.0   

                           aroon_timeperiod  ema  
BTCUSDT - 120m BEST_5  21                80   80  
                       27                80   90  
                       3                 80   50  
                       9                 80   60  
                       15                80   70  
BTCUSDT - 120m WORSE_3 12                50   70  
                       30                50  100  
                       0                 50   50

ITERATION n# 3 at 2022-08-14 03:27:51.316879. SYMBOL: BTCUSDT TIMEFRAME: 60m


...

pnl_net  won  lost  trades  moneydown_max  \
ETHUSDT - 60m BEST_5  24 -14.046850    2     6       8      18.404269   
                      30 -14.046850    2     6       8      18.404269   
                      18 -16.560966    1     8       9      20.918386   
                      12 -19.267776    1     9      10      23.625195   
                      6  -21.667904    1    10      11      26.025323   
ETHUSDT - 60m WORSE_3 4  -35.783262    0     8       8      37.970632   
                      10 -35.783262    0     8       8      37.970632   
                      16 -35.783262    0     8       8      37.970632   

                          long_won  long_lost  short_won  short_lost  \
ETHUSDT - 60m BEST_5  24         2          3          0           3   
                      30         2          3          0           3   
                      18         1          5          0           3   
                      12         1          6          0           3   
                      6          1          7          0           3   
ETHUSDT - 60m WORSE_3 4          0          3          0           5   
                      10         0          3          0           5   
                      16         0          3          0           5   

                          longs_pnl  shorts_pnl  leverage_factor  \
ETHUSDT - 60m BEST_5  24  -2.299190  -11.747660              1.0   
                      30  -2.299190  -11.747660              1.0   
                      18  -5.156923  -11.404043              1.0   
                      12  -8.233687  -11.034089              1.0   
                      6  -10.961852  -10.706051              1.0   
ETHUSDT - 60m WORSE_3 4  -10.038547  -25.744715              1.0   
                      10 -10.038547  -25.744715              1.0   
                      16 -10.038547  -25.744715              1.0   

                          aroon_timeperiod  ema  
ETHUSDT - 60m BEST_5  24                50   90  
                      30                50  100  
                      18                50   80  
                      12                50   70  
                      6                 50   60  
ETHUSDT - 60m WORSE_3 4                 90   50  
                      10                90   60  
                      16                90   70

ITERATION n# 19 at 2022-08-14 03:58:57.351365. SYMBOL: ETHUSDT TIMEFRAME: 30m


pnl_net  won  lost  trades  moneydown_max  \
ETHUSDT - 30m BEST_5  13  -6.586739    2     7       9      27.761511   
                      3   -9.480180    1     6       7      14.915278   
                      7   -9.688763    2     8      10      30.693622   
                      29  -9.753803    1     5       6      15.312972   
                      35  -9.753803    1     5       6      15.312972   
ETHUSDT - 30m WORSE_3 30 -25.878382    1    11      12      31.915905   
                      18 -25.198178    1    11      12      31.248639   
                      24 -25.198178    1    11      12      31.248639   

                          long_won  long_lost  short_won  short_lost  \
ETHUSDT - 30m BEST_5  13         2          6          0           1   
                      3          1          6          0           0   
                      7          2          7          0           1   
                      29         1          5          0           0   
                      35         1          5          0           0   
ETHUSDT - 30m WORSE_3 30         1          9          0           2   
                      18         1          8          0           3   
                      24         1          8          0           3   

                          longs_pnl  shorts_pnl  leverage_factor  \
ETHUSDT - 30m BEST_5  13  -3.749829   -2.836910              1.0   
                      3   -9.480180    0.000000              1.0   
                      7   -6.946060   -2.742703              1.0   
                      29  -9.753803    0.000000              1.0   
                      35  -9.753803    0.000000              1.0   
ETHUSDT - 30m WORSE_3 30 -20.770788   -5.107595              1.0   
                      18 -18.586707   -6.611471              1.0   
                      24 -18.586707   -6.611471              1.0   

                          aroon_timeperiod  ema  
ETHUSDT - 30m BEST_5  13                60   70  
                      3                 80   50  
                      7                 60   60  
                      29               100   90  
                      35               100  100  
ETHUSDT - 30m WORSE_3 30                50  100  
                      18                50   80  
                      24                50   90

ITERATION n# 20 at 2022-08-14 04:03:38.162884. SYMBOL: ETHUSDT TIMEFRAME: 15m


pnl_net  won  lost  trades  moneydown_max  \
ETHUSDT - 15m BEST_5  22 -11.584462    2    11      13      33.885085   
                      28 -13.475500    2    10      12      35.709695   
                      34 -13.475500    2    10      12      35.709695   
                      16 -14.140336    2    12      14      34.307779   
                      10 -19.194607    3    12      15      34.219173   
ETHUSDT - 15m WORSE_3 12 -41.880602    2    20      22      49.258813   
                      0  -39.769866    4    20      24      47.172057   
                      24 -38.055397    2    18      20      48.866363   

                          long_won  long_lost  short_won  short_lost  \
ETHUSDT - 15m BEST_5  22         2          6          0           5   
                      28         2          5          0           5   
                      34         2          5          0           5   
                      16         2         12          0           0   
                      10         3         12          0           0   
ETHUSDT - 15m WORSE_3 12         2         16          0           4   
                      0          3         18          1           2   
                      24         2         15          0           3   

                          longs_pnl  shorts_pnl  leverage_factor  \
ETHUSDT - 15m BEST_5  22   3.090739  -14.675201              1.0   
                      28   0.885826  -14.361327              1.0   
                      34   0.885826  -14.361327              1.0   
                      16 -14.140336    0.000000              1.0   
                      10 -19.194607    0.000000              1.0   
ETHUSDT - 15m WORSE_3 12 -29.581986  -12.298616              1.0   
                      0  -32.458521   -7.311346              1.0   
                      24 -28.228104   -9.827292              1.0   

                          aroon_timeperiod  ema  
ETHUSDT - 15m BEST_5  22                90   80  
                      28                90   90  
                      34                90  100  
                      16                90   70  
                      10                90   60  
ETHUSDT - 15m WORSE_3 12                50   70  
                      0                 50   50  
                      24                50   90

In [9]:
symbols_df.nlargest(5, columns='pnl_net')

pnl_net  won  lost  trades  moneydown_max  \
BNBUSDT - 120m BEST_5  13  101.742259    2     3       5      97.705501   
                       19  101.742259    2     3       5      97.705501   
DOGEUSDT - 120m BEST_5 33   94.385067    1     3       4     149.623088   
                       3    84.555305    1     3       4     108.832352   
                       9    84.555305    1     3       4     108.832352   

                           long_won  long_lost  short_won  short_lost  \
BNBUSDT - 120m BEST_5  13         2          1          0           2   
                       19         2          1          0           2   
DOGEUSDT - 120m BEST_5 33         1          1          0           2   
                       3          1          0          0           3   
                       9          1          0          0           3   

                            longs_pnl  shorts_pnl  leverage_factor  \
BNBUSDT - 120m BEST_5  13  128.509662  -26.767403              1.0   
                       19  128.509662  -26.767403              1.0   
DOGEUSDT - 120m BEST_5 33  100.789651   -6.404585              1.0   
                       3    91.819100   -7.263795              1.0   
                       9    91.819100   -7.263795              1.0   

                           aroon_timeperiod  ema  
BNBUSDT - 120m BEST_5  13                60   70  
                       19                60   80  
DOGEUSDT - 120m BEST_5 33                80  100  
                       3                 80   50  
                       9                 80   60